In [234]:
#uncomment if need to install dependencies

#!pip install pandas
#!pip install plotly
#!pip install plotly.express
#!pip install dash
#!pip install nbformat
#!pip install matplotlib

In [235]:
# load dependencies

import pandas as pd
import numpy as np
#import plotly.express as px
import plotly.graph_objects as go
import plotly.io as io
#io.renderers.default = 'browser' #to ensure prints as expect
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

from requests import get

In [236]:
# read tsv
filepath = '../data/raw/HRDataset_v14.xls.txt'
hr_df = pd.read_csv(filepath, sep='\t',  encoding='utf-16')

# read json
#json_url = 'https://raw.githubusercontent.com/christiebarron/dvc10-hr/refs/heads/main/data/raw/HRDataset_v14.json'
#hr_df = pd.DataFrame(get(json_url).json())

# save file as csv and json for easier reading
#hr_df.to_csv('../data/raw/HRDataset_v14.csv')
#hr_df.to_json('../data/raw/HRDataset_v14.json')


In [237]:
# check data
hr_df.columns

# continuous variable EDA
print(hr_df.dtypes) #data type
hr_df.describe() #continuous descriptives
len(hr_df.Employee_Name.unique()) #number of unique ids
hr_df.info



Employee_Name                  object
EmpID                           int64
MarriedID                       int64
MaritalStatusID                 int64
GenderID                        int64
EmpStatusID                     int64
DeptID                          int64
PerfScoreID                     int64
FromDiversityJobFairID          int64
Salary                          int64
Termd                           int64
PositionID                      int64
Position                       object
State                          object
Zip                             int64
DOB                            object
Sex                            object
MaritalDesc                    object
CitizenDesc                    object
HispanicLatino                 object
RaceDesc                       object
DateofHire                     object
DateofTermination              object
TermReason                     object
EmploymentStatus               object
Department                     object
ManagerName 

<bound method DataFrame.info of                 Employee_Name  EmpID  MarriedID  MaritalStatusID  GenderID  \
0         Adinolfi, Wilson  K  10026          0                0         1   
1    Ait Sidi, Karthikeyan     10084          1                1         1   
2           Akinkuolie, Sarah  10196          1                1         0   
3                Alagbe,Trina  10088          1                1         0   
4            Anderson, Carol   10069          0                2         0   
..                        ...    ...        ...              ...       ...   
306            Woodson, Jason  10135          0                0         1   
307        Ybarra, Catherine   10301          0                0         0   
308          Zamora, Jennifer  10010          0                0         0   
309               Zhou, Julia  10043          0                0         0   
310             Zima, Colleen  10271          0                4         0   

     EmpStatusID  DeptID  PerfS

In [238]:
# convert data to correct pandas type. 

# convert EmpID, ManagerID, deptid to string.
    # married, martial status, gender, emp status, fromdiv jobfair, recruitmentsource to unordered factor.
to_string = ['EmpID', 'MarriedID', 'DeptID', 'MarriedID', 'MaritalStatusID', 'GenderID', 'FromDiversityJobFairID', 'PositionID', 'ManagerID']
for var in to_string:
    hr_df[var] = hr_df[var].astype(str)

# performance score, satisfaction as ordered factor.
to_cat_ord = ['PerfScoreID', 'EmpSatisfaction']
for var in to_cat_ord:
    hr_df[var] = pd.Categorical(hr_df[var].astype(str), categories = ['1','2','3','4','5'], ordered = True)

# lastperformReview_date, DateofHire, DateofTermination as date object. 
to_date = ['LastPerformanceReview_Date', 'DateofHire', 'DateofTermination']
for var in to_date:
    hr_df[var] = pd.to_datetime(hr_df[var], format = '%m/%d/%Y')

hr_df['DOB'] = pd.to_datetime(hr_df['DOB'], format = '%m/%d/%y')
# note: dayslate and abscences are counts.


In [239]:
#hr_df.hist('Age')

#hr_df[hr_df['Age'] < 0][['Employee_Name', 'DOB', 'LastPerformanceReview_Date']]

In [240]:
# Set the date of data collection
date_of_data_collection = pd.to_datetime('2019-02-07')

# Calculate a proxy for age
hr_df['Age'] = (hr_df['LastPerformanceReview_Date'] - hr_df['DOB']) /pd.Timedelta(days=365.25)
#hr_df['Age'] = (date_of_data_collection - hr_df['DOB']) /pd.Timedelta(days=365.25)
hr_df['Age']

0      35.523614
1      40.807666
2      23.652293
3      30.266940
4      26.398357
         ...    
306    33.801506
307    33.330595
308    39.479808
309    39.937029
310    40.454483
Name: Age, Length: 311, dtype: float64

In [241]:
# filter data for typical data cleaning
hr_df
hr_df = hr_df[hr_df['Department'] != 'Executive Office'] # CIO

In [242]:
# Calculate BANs

def aggregate_var(df, column_name, var_type = 'continuous'):
    """
    Function that calculates the aggregation of a continuous or categorical variable for the BANs
    """

    res_list = []

    if var_type == 'continuous':
        res_list = df[column_name].mean(),df[column_name].median()
    elif var_type == 'categorical':
        length_no_miss = df[column_name].notna().sum()
        res_list = np.round((df[column_name].isin(['4', '5']).sum() / length_no_miss) * 100, 2), np.round((df[column_name].isin(['1','2','3']).sum() / length_no_miss) * 100, 2)
    elif var_type == 'count':
        res_list = df[column_name].notna().sum()

    return(res_list)

#count
EmpIDRes = len(hr_df.EmpID.unique())
#cont
EngagementSurveyRes = aggregate_var(df = hr_df, column_name = 'EngagementSurvey', var_type = 'continuous')
SalaryRes = aggregate_var(df = hr_df, column_name = 'Salary', var_type = 'continuous')
#cat
EmpSatisfactionRes = aggregate_var(df = hr_df, column_name = 'EmpSatisfaction', var_type = 'categorical')
PerfScoreIDRes = aggregate_var(df = hr_df, column_name = 'PerfScoreID', var_type = 'categorical')




In [243]:
# get unique values for department
dept_select = hr_df.Department.unique()
position_select = hr_df.Position.unique() #consider not providing executive positions here.

## Data Visualization


### Descriptive Visualizations

In [247]:
# gender pie

def gender_pie_fig(hr_df):

    gender_df = hr_df.groupby(by = 'GenderID').size().reset_index(name = 'Count')
    gender_total_counts = gender_df['Count'].sum()
    gender_df['Percentage'] = ((gender_df['Count']/gender_total_counts)*100).round(1)#.astype(str) + '%'
    gender_pie_fig = px.pie(gender_df, values = 'Count', names = 'GenderID', hole = .6, title = "Gender Composition",  
        color_discrete_sequence=[px.colors.qualitative.Pastel[5], px.colors.qualitative.Pastel[3]]
        )
    return(gender_pie_fig)

gender_pie_fig(hr_df)



### Satisfaction Visualizations

In [248]:
# Create the stacked bar chart

# to-do: convert the 1:5 to meaningful values for EmpSatisfaction

def satisfaction_stacked_fig(hr_df, 
                             filter_var = ['Department', 'EmpSatisfaction'],
                             group_var = ['Department']):

    # dynamic variables
    #filter_var = ['Department', 'EmpSatisfaction']
    #group_var = ['Department']

    #data wrangling
    satis_df_long = hr_df[filter_var].melt(id_vars = group_var, var_name = 'Question', value_name = 'Response')
    satis_df_clean = satis_df_long.groupby(by = ['Department', 'Question', 'Response']).size().reset_index(name = 'Count')
    total_counts = satis_df_clean.groupby(['Department', 'Question'])['Count'].transform('sum')
    satis_df_clean['Percentage'] = ((satis_df_clean['Count']/total_counts)*100).round(1)#.astype(str) + '%'
    satis_df_clean['Label'] = satis_df_clean['Percentage'].astype(str) + '%'
    #color
    satis_color = [px.colors.sequential.RdBu[i] for i in [2, 4, 5, 7, 9]]
    satis_color[2] = 'rgb(211, 211, 211)'

    #create figure
    satis_fig = px.bar(satis_df_clean, y='Department', x='Percentage', color='Response', title='Employee Satisfaction', 
                    color_discrete_sequence = satis_color,
                    text = 'Label', #add percent labels to the graph
                        template='plotly_white') #make the background white, not default light blue

    satis_fig.update_traces(texttemplate='%{text}', textposition='inside', insidetextanchor='middle')
    # Update the layout to make it a stacked bar chart
    satis_fig.update_layout(barmode='stack', xaxis_title='Percent of Responses', yaxis_title='Department', legend_title='Responses'
                    #remove background color
                    #plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)'
    )
    satis_fig.update_xaxes(showgrid=False)
    satis_fig.update_yaxes(showgrid=False)

    # Show the figure
    #satis_fig.show()

    return(satis_fig)

satisfaction_stacked_fig(hr_df)



/var/folders/bm/zz3zwwtj491ggdvwhnvk_wyr0000gn/T/ipykernel_919/2136305100.py:15: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [250]:
# mean engagement by department

def engagement_bar_fig(hr_df):

    engage_df = hr_df.groupby('Department')[['EngagementSurvey']].mean().reset_index().round(1)

    engage_fig = px.histogram(engage_df, x = 'EngagementSurvey', y = 'Department', title = 'Engagement by Department',
                            template='plotly_white',
                            #text = 'EngagementSurvey'
                            )
    engage_fig.update_layout(xaxis_title = 'Engagement')
    engage_fig.update_traces(marker_color = px.colors.sequential.RdBu[7])
    engage_fig.update_traces(text=engage_df['EngagementSurvey'], textposition='outside') #, marker=dict(color='blue', line=dict(color='darkblue', width=1.5)))
    #egage_fig.update_traces(marker = dict(color = satis_color[3]), line = dict(color = 'darkblue', width = 1.0))
    #egage_fig.show()
    return(engage_fig)

engagement_bar_fig(hr_df)

### Performance 

### Exploratory Data Analysis

In [ ]:
hr_df

### Planning

- BANs: satisfaction, performance, engagement, pay

     - Categorical Group vars: department, employment status, position, voluntarily left 

     - continuous group/filter vars: DateofHire

#### Potential Dashboard Design

Some great dashboard examples: 
- https://public.tableau.com/app/profile/pradeepkumar.g/viz/HRAttritionDashboardRWFD_16570446563570/viz 
- https://public.tableau.com/app/profile/gandes.goldestan/viz/HRDashboard_16284874251120/Overview
- https://public.tableau.com/app/profile/frankie.benson/viz/HRDashboard-RWFD_16791683724400/HRExecutiveSummary 

#### Potential Research Questions

- RQ 1: how are different departments doing regarding employee performance, engagement, satisfaction, Special Projects, Days Late (controlling for pay)?
     - among employees currently in company
- RQ 2: who are managers with above and beyond scores controlling for department, employee pay.
- RQ3: how does employee performance vary based on Recruitment source? (controll for department, pay)
- RQ4: if data longitudinal, what are the trajectories of performance over last 4 quarters?
